# Projeto 2 - Fine-Tuning de um modelo transformer para classificação de sentimento

In [8]:
%pip install -q -U watermark

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install -q transformers==4.46.3
# %pip install -q scikit-learn==1.5.2
# %pip install -q tokenizers==0.20.3
# %pip install -q numpy==1.26.2
# %pip install -q tqdm==4.65.0
# %pip install -q tensorflow==2.18.0
# %pip install -q keras==3.7.0
# %pip install -q spacy==3.7.2
# %pip install -q scipy==1.11.1
# %pip install -q nltk==3.8.1
# %pip install -q pandas==2.2.3
# %pip install -q keras-preprocessing==1.1.2
# %pip install -q matplotlib==3.7.4

In [9]:
# %watermark --iversions

In [162]:
# Imports

import math
import nltk
import spacy
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import transformers
from tokenizers import BertWordPieceTokenizer
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import load_model
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.metrics import Precision, Recall, AUC
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, LearningRateScheduler, CallbackList, ReduceLROnPlateau
from keras.optimizers import Adam


In [11]:
%reload_ext watermark
%watermark -a "Angelo Medeiros Nóbrega"

Author: Angelo Medeiros Nóbrega



# Carregando os dados de texto

In [12]:
# Carrega os dados de treino
dados_treino = pd.read_csv("dados_treino.txt", header=None, delimiter=";")

In [13]:
# Carrega os dados de teste
dados_teste = pd.read_csv("dados_teste.txt", header=None, delimiter=";")

In [14]:
# Ajusta os nomes das colunas
dados_treino = dados_treino.rename(columns={0: "texto", 1: "sentimento"})
dados_teste = dados_teste.rename(columns={0: "texto", 1: "sentimento"})

In [15]:
# Shape - Dados de treino
dados_treino.shape

(16000, 2)

In [16]:
# Shape - Dados de teste
dados_teste.shape

(2000, 2)

In [17]:
# Amostra de teste
dados_teste.head()

,texto,sentimento
0,i feel like my only role now would be to tear ...,sadness
1,i feel just bcoz a fight we get mad to each ot...,anger
2,i feel like reds and purples are just so rich ...,joy
3,im not sure the feeling of loss will ever go a...,sadness
4,i feel like ive gotten to know many of you thr...,joy


In [18]:
# Amostra de treino
dados_treino.head()

,texto,sentimento
0,i am feeling completely overwhelmed i have two...,fear
1,i have the feeling she was amused and delighted,joy
2,i was able to help chai lifeline with your sup...,joy
3,i already feel like i fucked up though because...,anger
4,i still love my so and wish the best for him i...,sadness


In [19]:
# Sentimentos presentes nos dados de treino
dados_treino["sentimento"].value_counts()

sentimento
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

In [20]:
# Sentimentos presentes nos dados de teste
dados_teste["sentimento"].value_counts()

sentimento
joy         695
sadness     581
anger       275
fear        224
love        159
surprise     66
Name: count, dtype: int64

# Pré-processamento dos dados de texto com SpaCy

In [21]:
!python -m spacy download en_core_web_md -q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [49]:
# Carrega o dicionário
nlp = spacy.load("en_core_web_md")

In [23]:
# Definição da função `preprocessamento_texto` que recebe um texto como parâmetro
def preprocessamento_texto(text):
  # Processa o texto usando o dicionário
  doc = nlp(text)

  # Cria umalista de lemas dos tokens, convertidos para minúsculas e sem espaços em branco,
  # excluindo as palabras que são stopwords
  tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop]

  # Retorna os tokens processados como uma string única, unindo-os com espaços
  return ' '.join(tokens)

In [24]:
# Aplica a função em treino
dados_treino['texto_processado'] = dados_treino['texto'].apply(preprocessamento_texto)

In [25]:
# Aplica a função em teste
dados_teste['texto_processado'] = dados_teste['texto'].apply(preprocessamento_texto)

In [26]:
# Amostra dos dados de treino
dados_treino.head()

,texto,sentimento,texto_processado
0,i am feeling completely overwhelmed i have two...,fear,feel completely overwhelmed strategy help feel...
1,i have the feeling she was amused and delighted,joy,feeling amuse delight
2,i was able to help chai lifeline with your sup...,joy,able help chai lifeline support encouragement ...
3,i already feel like i fucked up though because...,anger,feel like fuck not usually eat morning
4,i still love my so and wish the best for him i...,sadness,love wish good long tolerate effect bm life fa...


# Versao 1 do modelo - Arquitetura Fully Connected neural network

Vetorização serve para converter os dados de entrada em uma representação numérica. Ideal para textos muito simples onde o contexto não é tão relevante. Calcula a frequência de cada palavra em cada texto e cria um vetor com essas frequências.

Desvantagens do TF-IDF:
- Não consegue capturar o contexto;

## Passo 1: Vetorização com TF-IDF

In [50]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [91]:
# Aplicar o vetorizador
dados_treino_tfidf = tfidf.fit_transform(dados_treino['texto_processado'])
dados_teste_tfidf = tfidf.transform(dados_teste['texto_processado'])

In [110]:
dados_treino_tfidf.shape

(16000, 5586)

In [113]:
dados_teste_tfidf.shape

(2000, 5586)

In [115]:
type(dados_treino_tfidf)

scipy.sparse._csr.csr_matrix

In [116]:
type(dados_teste_tfidf)

scipy.sparse._csr.csr_matrix

In [150]:
# Converte os dados de entrada (text) para array
X_treino_array = dados_treino_tfidf.toarray()
X_teste_array = dados_teste_tfidf.toarray()

type(X_treino_array)

numpy.ndarray

## Passo 2: Preparaçào dos dados


Precisamos agora converter a variável alvo para representaçào numérica. Para isso, vamos usar a função LabelEncoder do scikit-learn.

In [118]:
# Cria o label encoder
le = LabelEncoder()

In [120]:
# Faz o fit e transform na variável alvo em treino
y_treino_le = le.fit_transform(dados_treino['sentimento'])

In [121]:
# Faz o transform na variável alvo em teste
y_teste_le = le.transform(dados_teste['sentimento'])

Vamos tratar de forma automática o desbalanceamento de classe

In [122]:
# Peso das classes
pesos_classes = compute_class_weight('balanced', classes=np.unique(y_treino_le), y=y_treino_le)

In [131]:
np.unique(y_treino_le)

array([0, 1, 2, 3, 4, 5])

In [123]:
type(pesos_classes)

numpy.ndarray

In [136]:
print(pesos_classes)

[1.23513973 1.37669936 0.49732687 2.04498978 0.57151022 4.66200466]


### Validação

In [146]:
# Divisão em dados de treino e teste (validação)
X_treino, X_val, y_treino, y_val = train_test_split(X_treino_array, y_treino_le, test_size=0.2, random_state=42, stratify=y_treino_le)

numpy.ndarray

In [140]:
# Ajusta a variável alvo como tipo categórica
y_treino_encoded = to_categorical(y_treino)
y_teste_encoded = to_categorical(y_teste_le)
y_val_encoded = to_categorical(y_val)

In [142]:
# Shapes
y_treino_encoded.shape, y_teste_encoded.shape, y_val_encoded.shape

((12800, 6), (2000, 6), (3200, 6))

# Passo 3: Construção do modelo

In [170]:
# Cria o modelo

# Inicializa um modelo sequencial. Modelos sequencias sao uma pilha linear de camadas.
model_v1 = Sequential()

# Define o número de colunas do X_treino
numero_de_colunas_X_treino = X_treino.shape[1]

# Adiciona a primeira camada densa (Fully-connected) ao modelo
model_v1.add(
    Dense(
        4096,
        # Utiliza a funcao de ativacao SELU (Scaled Exponential Linear Unit)
        activation='selu',

        # Inicializa os pesos com a distribuicao LeCun Normal
        kernel_initializer='lecun_normal',

        # Define o formato da entrada com base no número de features do X_treino
        input_shape=(numero_de_colunas_X_treino,),

        # Aplica regularização L2 para reduzir o overfitting
        kernel_regularizer=tf.keras.regularizers.l2(0.01)
    )
)

# Adiciona a segunda camada densa
model_v1.add(
    Dense(
        2048,
        activation='selu',
        kernel_initializer='lecun_normal',
        kernel_regularizer=tf.keras.regularizers.l2(0.01)
    )
)

# Adiciona a terceira camada densa
model_v1.add(
    Dense(
        1024,
        activation='selu',
        kernel_initializer='lecun_normal',
        kernel_regularizer=tf.keras.regularizers.l2(0.01)
    )
)

# Adiciona a quarta camada densa
# Camada com 64 neurônios e ativacao SELU
model_v1.add(
    Dense(
        64,
        activation='selu',
    )
)

# Adiciona a camada de saída
# Camada com 6 neurônios e ativacao softmax para classificação multiclasse
model_v1.add(Dense(6, activation='softmax'))

# O número de neuronios de cada camada é definido a partir de testes e experimentações.
# Não existe uma regra fixa para definir o número de neurônios de cada camada.
# O número de camadas e neurônios é um hiperparâmetro que deve ser ajustado de acordo com o problema.

c:\Users\angel\anaconda3\envs\dsa-formacao-ia\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
